In [1]:
from data_sci_toolkit.aws_tools import redshift_tools

In [2]:
df_ep_train = redshift_tools.get_event_propensity_training_dataset(
    cluster= "prod-app",
    database= "stlrcanucks",
    lkupclientid= 7,
    start_year= 2010,
    end_year= 2021
)

df_ep_train.shape

Authorized as AROASQ4JELIXYLYV6P4UV:rkazmerik@stellaralgo.com


(254194, 16)

In [6]:
df_ep_train.did_purchase.value_counts()

1    127097
0    127097
Name: did_purchase, dtype: int64

In [21]:
df_ep_scoring = redshift_tools.get_event_propensity_scoring_dataset(
    cluster= "prod-app",
    database= "stlrcanucks",
    lkupclientid= 7,
    game_date= "2022-10-28"
)

df_ep_scoring.shape

Authorized as AROASQ4JELIXYLYV6P4UV:rkazmerik@stellaralgo.com


(626304, 12)

In [8]:
df_ep_scoring.head(3)

,daysout,eventdate,tenure,dimcustomermasterid,events_purchased,frequency_opponent,frequency_eventday,frequency_eventtime,inmarket,distancetovenue,recent_openrate,recent_clickrate
0,Day Of,2022-10-28,2594,573252,2,0.0000,0.0000,0.0084,True,26.4952,None,None
1,1 to 3 Days Out,2022-10-28,2594,573252,2,0.0000,0.0000,0.0084,True,26.4952,None,None
2,4 to 7 Days Out,2022-10-28,2594,573252,2,0.0000,0.0000,0.0084,True,26.4952,None,None


In [13]:
df_ep_scoring.events_purchased.value_counts(bins=[0,3,5,10,100,1000])

(-0.001, 3.0]      1196612
(3.0, 5.0]           42512
(5.0, 10.0]          25384
(10.0, 100.0]        12560
(100.0, 1000.0]         64
Name: events_purchased, dtype: int64